In [1]:
#Librerias utilizadas
from finrisk import QC_Financial_3 as Qcf
from typing import List, Tuple, Callable
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

#from modules
import auxiliary as aux
from numpy import random as rnd
import plotly.express as px
import pandas as pd
import numpy as np
import math
from scipy.stats import norm

# Tarea 2 Carlos Quintana - Daniel García-vinuesa

- Usando las funciones anteriores, o desarrollando las propias, escriba una función que pueda valorizar, por simulación de Montecarlo, calls y puts sobre bonos cero cupón con el modelo de HW. Considere puts y calls de plazo máximo igual a 1Y.
- Su función debe poder recibir la semilla.
- Compare sus resultados con las fórmulas del modelo y muestre que diferencias obtiene usando 1000 simulaciones y una semilla predeterminada.
- Valorice también por simulación de Montecarlo el bono cupón cero a 1Y. Muestre que diferencias obtiene respecto al valor de mercado del bono usando 1000 simulaciones y una semilla predeterminada.


# Data

In [2]:
#Data
curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [3]:
curva['t'] = curva['plazo'] / 365.0

In [4]:
curva['rate'] = np.log(1 / curva['df'])/( curva['plazo'] / 365.0)

In [5]:
zcurva = interp1d(curva['t'],
                  curva['rate'],
                  kind='cubic',
                  fill_value="extrapolate")

In [6]:
# acá se define la función fwd y dfwd
def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

In [7]:
def zrate(t: float) -> float:
    return zcurva(t)


def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta


def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

# Parámetros 1


In [8]:
#Parámetros
sigma = 0.015
gamma = 0.5
r = zrate(0)
t = 0
T_opcion=1
T_bono=2
k=1.01
num_sim = 1000
num_steps = 264
seed = 1234
df = 0
dt=1/264

# Función tetha, b_hw, a_hw y zero_hw

In [9]:
def theta(t: float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

In [10]:
def b_hw(gamma: float, t: float, T: float) -> float:
    """
    Calcula el valor de la función B(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    
    params:
    
    - gamma: intensidad de reversión del modelo HW
    - t:
    - T:
    
    return:
    
    - valor de la función B(t, T)
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

In [11]:
def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float,
         verbose = False):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3


In [12]:
def zero_hw(r: float, gamma: float, sigma: float, zrate: float, fwd: float, t: float, T: float) -> float:
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)


# Funciónes de Call y Put cerradas

In [13]:
#Funciones

#Función sz 
def sz(T_opcion,T_bono,gamma,sigma):
    b=b_hw(gamma,T_opcion,T_bono)
    sz=b*np.sqrt((sigma**2/(2*gamma))*(1-np.exp(-2*gamma*T_opcion)))
    return sz

#Función d1
def d1(T_bono,T_opcion,t,sigma,gamma,r,k):
    z_op=zero_hw(r,gamma,sigma, zrate,fwd,t,T_opcion)
    z_bn=zero_hw(r,gamma,sigma, zrate,fwd,t,T_bono)
    s_z=sz(T_opcion,T_bono,gamma,sigma)
    d1=(np.log(z_bn/(k*z_op)+(0.5*s_z**2))/s_z)
    return d1

#Función d2  
def d2(T_bono,T_opcion,t,sigma,gamma,r,k):
    z_op=zero_hw(r,gamma,sigma, zrate,fwd,t,T_opcion)
    z_bn=zero_hw(r,gamma,sigma, zrate,fwd,t,T_bono)
    s_z=sz(T_opcion,T_bono,gamma,sigma)
    d_1=d1(T_bono,T_opcion,t,sigma,gamma,r,k)
    d2=d_1-s_z
    return d2

#Funciones para call y put
def call(T_bono,T_opcion,t,sigma,gamma,r,k):
    z_op=zero_hw(r,gamma,sigma, zrate,fwd,t,T_opcion)
    z_bn=zero_hw(r,gamma,sigma, zrate,fwd,t,T_bono)
    d_1=d1(T_bono,T_opcion,t,sigma,gamma,r,k)
    d_2=d2(T_bono,T_opcion,t,sigma,gamma,r,k)
    call=z_op*((z_bn/z_op)*norm.cdf(d_1)-k*norm.cdf(d_2))
    return call

def put(T_bono,T_opcion,t,sigma,gamma,r,k):
    z_op=zero_hw(r,gamma,sigma, zrate,fwd,t,T_opcion)
    z_bn=zero_hw(r,gamma,sigma, zrate,fwd,t,T_bono)
    d_1=d1(T_bono,T_opcion,t,sigma,gamma,r,k)
    d_2=d2(T_bono,T_opcion,t,sigma,gamma,r,k)
    put=z_op*((z_bn/z_op)*-norm.cdf(-d_1)+k*norm.cdf(-d_2))
    return put

#Resultados con parámetros 1
print("La Put es:",put(T_bono,T_opcion,t,sigma,gamma,r,k))   
print("La Call es:",call(T_bono,T_opcion,t,sigma,gamma,r,k)) 

La Put es: 0.011063878585398887
La Call es: 0.0006385183204811755


# Simulación del discount factor (ez) y r

In [14]:
#Función theta que depende de t y T
def theta_fwd_measure(t: float, T:float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux - b_hw(gamma, t, T)*sigma**2

In [15]:
#Función que realiza las simulaciones
def sim_hw_many(gamma, sigma, theta_fwd_measure,T, r0, num_sim, num_steps, seed,dt):
    """
    """
    
    num_steps += 1
    
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    np.random.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = np.random.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    for i in range(1, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta_fwd_measure(i * dt,T)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return sim

In [16]:
#Resultado de la simulación  de r
s=sim_hw_many(gamma, sigma, theta_fwd_measure,T_opcion, r, num_sim, num_steps, seed,dt)
r_prom=np.mean(s)
print("El promedio de las simulaciones es:",r_prom)

El promedio de las simulaciones es: 0.0003359968463563494


**AD:** revisa valores de variables.

In [17]:
print(f'num_sim: {num_sim}')
print(f'num_steps: {num_steps}')
print(f'seed: {seed}')

num_sim: 1000
num_steps: 264
seed: 1234


## Función que calcula el factor de descuento

In [18]:
# Función que calcula el ez y z curva, además de su diferencia absoluta
def factor_dct(s,df,curva,dt):
    for sim in s:
        df += math.exp(-dt * np.sum(sim))

    ez = df / num_sim
    z_curva = curva['df'].iloc[15]
    return (f"ez: {ez: .8%}"), (f"z_curva: {z_curva: .8%}"),(f"diferencia: {abs(z_curva-ez): .8}")
factor_dct(s,df,curva,dt)

('ez:  99.96894692%', 'z_curva:  99.92978688%', 'diferencia:  0.00039160039')

**AD:** están chequeando el factor de descuento en la medida forward. Esto no es correcto, para esta verificación hay que usar la medida libre de riesgo.

# Parámetros 2 (para calcular call y put con simulación)

In [19]:
# Parámetros para la simulación
s = sim_hw_many(gamma, sigma, theta_fwd_measure,
                T_opcion, r, num_sim, num_steps, seed, dt)
tasas = s[:, -1]
num_steps = 263

## Función para el payoff de la call y put

In [20]:
# Funciones para la call y put, las cuales son utilizadas en la función "Simulacion"
def payoff_call(z, k):
    C = max(z-k, 0)
    return C


def payoff_put(z, k):
    P = max(-z+k, 0)
    return P

## Simulación de la call y la put

In [21]:
# Simulación Call y Put- Payoff
def Simulación(gamma, sigma, zrate, fwd, theta_fwd_measure, T_opcion, T_bono, r, num_sim, num_steps, seed, k, s):
    z = []
    sim_call = []
    sim_put = []
    i = 0
    # Condición del loop
    while i < len(tasas):
        # variable que contiene la función zerp_hw para las diferentes tasas utilizadas
        zero = zero_hw(tasas[i], gamma, sigma, zrate, fwd, T_opcion, T_bono)
        z.append(zero)
        # Payoff call y put
        C = payoff_call(z[i], k)
        sim_call.append(C)
        P = payoff_put(z[i], k)
        sim_put.append(P)
        i = i+1

    return sim_call, sim_put

 ### Función que calcula el factor de descuento para la simulación

In [22]:
#función que retorna el factor de descuento, utilizando parámetros 2
def disc_fac(tasas,dt):
    df=0
    for i in tasas:
        df += math.exp(-dt * np.sum(i))
    discount_factor=df/len(tasas)
    print(f"ez: {discount_factor: .8%}")
    return discount_factor

### Función que calcula los valores de call y put

In [23]:
# Función que retorna los valroes de la call y de la put hechas con simulación, además de su tasa de descuento
def payoff(tasas, gamma, sigma, zrate, fwd, theta_fwd_measure, T_opcion, T_bono,
           r, num_sim, num_steps, seed, k, s):
    call, put = Simulación(gamma, sigma, zrate, fwd, theta_fwd_measure,
                           T_opcion, T_bono, r, num_sim, num_steps, seed, k, s)
    c = []
    p = []
    discount_factor = disc_fac(tasas, dt)
    j = 0
    while j < len(call):
        call_s = call[j]*discount_factor
        c.append(call_s)
        put_s = put[j]*discount_factor
        p.append(put_s)
        j = j+1
    return discount_factor, np.average(c), np.average(p)

In [24]:
payoff(tasas,gamma,sigma,zrate,fwd,theta_fwd_measure,T_opcion,T_bono,r,num_sim,num_steps,seed,k,s)

ez:  99.99987278%


(0.9999987277931004, 0.0005638567882788321, 0.010777615395188381)

# Comparativa entre fórmulas cerradas y simulación

In [25]:
#Comparación de resultados con simulación y fórmula cerrada
disc,call_sim, put_sim=payoff(tasas,gamma,sigma,zrate,fwd,theta_fwd_measure,T_opcion,T_bono,r,num_sim,num_steps,seed,k,s)
#Comparación de la Put
print("La Put con fórmula cerrada es:",put(T_bono,T_opcion,t,sigma,gamma,r,k))   
print("La Put calculada con simulación es:",put_sim)  

print("=============================================================")

#Comparación de la Call
print("La Call con fórmula cerrada es:",call(T_bono,T_opcion,t,sigma,gamma,r,k)) 
print("La Call calculada con simulación es:",call_sim)  

ez:  99.99987278%
La Put con fórmula cerrada es: 0.011063878585398887
La Put calculada con simulación es: 0.010777615395188381
La Call con fórmula cerrada es: 0.0006385183204811755
La Call calculada con simulación es: 0.0005638567882788321


## Test

In [26]:
import hull_white as hw

In [32]:
call = hw.zcb_call_put(hw.CallPut.CALL, k, T_opcion, T_bono, zrate(0),
                       math.exp(-zrate(T_opcion) * T_opcion),
                       math.exp(-zrate(T_bono) * T_bono),
                       gamma,
                       sigma)
print(k, call)

1.01 0.00063851832291556


In [33]:
put = hw.zcb_call_put(hw.CallPut.PUT, k, T_opcion, T_bono, zrate(0),
                       math.exp(-zrate(T_opcion) * T_opcion),
                       math.exp(-zrate(T_bono) * T_bono),
                       gamma,
                       sigma)
print(k, put)

1.01 0.01106387858783342
